#  Market Basket Analysis with Apriori Algorithm

Sugestão de produtos aos usuários. Neste estudo, aplicaremos a análise Market Basket usando o algoritmo Apriori. 

As regras de Associação têm como premissa básica encontrar elementos que implicam na presença de outros elementos em uma mesma transação, ou seja, encontrar relacionamentos ou padrões frequentes entre conjuntos de dados.

**Conjunto de dados:** Foi utilizado o conjunto de dados Online Retail II, que inclui os dados de vendas da loja de vendas online sediada no Reino Unido.

### Variables Descriptions:

   - BillNo: bill number -> operation.
   - Itemname: Product name
   - Quantity: Number of products
   - Date
   - Price
   - CustomerID: Unique customer number
   - Country
   - **produto_id -> foi criado essa coluna no dataset** 

#### Métricas 
   - **support:**
   - **suporte antecedente** 
   - **suporte conseqüente** 
   - **confiança** 
   - **lift** 

## Referencias:
http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

http://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/

# 0.0. Imports

In [88]:
import numpy    as np
import pandas   as pd
import datetime as dt
import pickle
import inflection
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# 0.1. Function

In [2]:
 def data_preparation( dataframe ):
    # Rename Columns  
    cols_old = ['BillNo', 'ItemName', 'Quantity', 'Date', 'Price', 'CustomerID', 'Country']

    snakecase = lambda x: inflection.underscore( x )
    cols_news = list( map( snakecase, cols_old ) )

    # Rename
    dataframe.columns = cols_news
    
    # Drop NA
    dataframe = dataframe.dropna( subset=['item_name','customer_id'] )
    
    # Data Types
    #dataframe['customer_id'] = dataframe['customer_id'].astype(int)
    #dataframe['bill_no'] = dataframe['bill_no'].astype(int)
    
    #feature_engineering
    dataframe = dataframe.loc[dataframe['price'] >= 0.04,:]

    dataframe = dataframe[~dataframe["item_name"].str.contains( "POST", na=False )]
    
    dataframe = dataframe[~dataframe['country'].isin( ["Unspecified"] )]

    dataframe = dataframe[~dataframe['customer_id'].isin( [16446] )]
    
    # removendo hora
    #dataframe['date'] = dataframe['date'].apply( get_month )
    # month
    #dataframe['month'] = dataframe['date'].dt.month

    # data product_id -> criando codigo unico para os produtos.
    df_product_id = dataframe.drop( ['bill_no', 'quantity', 'date', 'price','customer_id', 'country'], axis=1 ).drop_duplicates( ignore_index=True )
    df_product_id = pd.DataFrame( df_product_id ) 
    df_product_id['produto_id'] = pd.factorize( df_product_id['item_name'])[0]

    # merge produto_id com dataframe
    dataframe = pd.merge( dataframe, df_product_id, on='item_name', how='left' )
    
    return dataframe

**Função data_preparation foi construida a partir do dataset data-exploration, após a análise dos dados e entendimento de algumas Premissas de Negócios**

# 1.0. Loading Data

In [3]:
df_raw = pd.read_excel( '../data/raw/DataSet_Test.xlsx', usecols="A:G" )

In [4]:
df1 = df_raw.copy()

# 2.0. Data preparation and Feature Engineering

In [5]:
df1 = data_preparation( df1 )
df1.head()

,bill_no,item_name,quantity,date,price,customer_id,country,produto_id
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0
1,536365,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,1
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,3
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,4


**Coluna produto_id foi adicionada no dataset**

# 3.0 Filtragem Country

In [6]:
def data_filter( dataframe, country=False, Country=""):
    if country:
        dataframe = dataframe[dataframe["country"] == Country]
    return dataframe

In [7]:
df_country = data_filter( df1, True, 'France' )
df_country.head()

,bill_no,item_name,quantity,date,price,customer_id,country,produto_id
26,536370,ALARM CLOCK BAKELIKE PINK,24,2010-12-01 08:45:00,3.75,12583.0,France,26
27,536370,ALARM CLOCK BAKELIKE RED,24,2010-12-01 08:45:00,3.75,12583.0,France,27
28,536370,ALARM CLOCK BAKELIKE GREEN,12,2010-12-01 08:45:00,3.75,12583.0,France,28
29,536370,PANDA AND BUNNIES STICKER SHEET,12,2010-12-01 08:45:00,0.85,12583.0,France,29
30,536370,STARS GIFT TAPE,24,2010-12-01 08:45:00,0.65,12583.0,France,30


**Vamos lidar com os dados de vendas da France como exemplo** 

# 4.0 Preparando a Matriz Compra-Produto

In [8]:
def create_purchase_product( dataframe, id=False ):
    if id:
        return dataframe.groupby(['bill_no', 'produto_id'])['quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['bill_no', 'item_name'])['quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)

In [35]:
purchase_product = create_purchase_product( df_country, id=True)
purchase_product.head()

produto_id,0,3,4,5,7,9,10,11,12,15,...,3781,3782,3784,3785,3786,3816,3817,3818,3820,3821
bill_no,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Coluna produto_id foi criada para realizar a contrução dessa Matriz, associar compras por codigo do produto**

## 4.1 Check prouto_id

In [37]:
def check_produto_id( dataframe, produto_id ): 
    item_name = dataframe[ dataframe["produto_id"] == produto_id]["item_name"].unique()[0] 
    return produto_id, item_name 

In [38]:
check_produto_id( df_country, 34 )

(34, 'ROUND SNACK BOXES SET OF4 WOODLAND')

In [39]:
check_produto_id( df_country, 244 )

(244, 'SET OF 6 T-LIGHTS SANTA')

**Função para encontrar o produto através do código ID**

# 5.0 Geração de regras de associação a partir de itens frequentes

**Vamos retornar os itens e conjuntos de itens com pelo menos 60% de suporte**

In [64]:
frequent_itemsets = apriori( purchase_product, min_support=0.06, use_colnames=True )
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.sort_values('support', ascending=False ).sample(10)

/opt/anaconda/envs/manfing/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets,length
65,0.067708,"(65, 258)",2
26,0.072917,(319),1
36,0.070312,(549),1
12,0.156250,(65),1
75,0.065104,"(26, 27, 28)",3
10,0.106771,(44),1
72,0.091146,"(685, 422)",2
74,0.065104,"(2902, 2903)",2
40,0.070312,(920),1
14,0.145833,(67),1


 **support:** Indica com que frequência o conjunto de itens ocorre no conjunto de dados

In [41]:
asso_rules = association_rules( frequent_itemsets, metric="lift", min_threshold=0.7 )
asso_rules.sort_values( 'support', ascending=False ).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
13,(171),(64),0.140625,0.130208,0.125000,0.888889,6.826667,0.106689,7.828125
12,(64),(171),0.130208,0.140625,0.125000,0.960000,6.826667,0.106689,21.484375
26,(422),(423),0.138021,0.174479,0.106771,0.773585,4.433681,0.082689,3.646050
27,(423),(422),0.174479,0.138021,0.106771,0.611940,4.433681,0.082689,2.221254
21,(171),(170),0.140625,0.135417,0.104167,0.740741,5.470085,0.085124,3.334821
30,(685),(423),0.171875,0.174479,0.104167,0.606061,3.473541,0.074178,2.095553
31,(423),(685),0.174479,0.171875,0.104167,0.597015,3.473541,0.074178,2.054977
11,(170),(64),0.135417,0.130208,0.104167,0.769231,5.907692,0.086534,3.769097
10,(64),(170),0.130208,0.135417,0.104167,0.800000,5.907692,0.086534,4.322917
20,(170),(171),0.135417,0.140625,0.104167,0.769231,5.470085,0.085124,3.723958


### Métricas que vemos na tabela acima:



   **Antecedents:** Se X é chamado de antecedente, 'suporte antecedente' calcula a proporção de transações que contêm o antecedente X.
   
   **Consequents:** se Y for chamado de conseqüente, 'suporte conseqüente' calcula a proporção de transações que contêm o antecedente Y.
   
   **Confidence:** indicando % dos clientes que compram produto X também compram Y ao mesmo tempo. 
   
   **Lift:** Quanto maior lift, mais forte é a associação entre o antecedente e o consequente. 
    
   **Conviction:** Razão da frequência esperada que X ocorre sem Y. uma alta convicção significa que o consequente é altamente dependente do antecedente.

In [35]:
print( f'Total de associação encontrado: {len(asso_rules)}') 

Total de associação encontrado: 52


# 6.0 Recommendation Product Customer

In [36]:
sorted_rules = asso_rules.sort_values( "lift", ascending=False )

In [37]:
produto_id = 170

check_produto_id( df_country, produto_id )

(170, 'SET/20 RED RETROSPOT PAPER NAPKINS')

In [38]:
product_id = 170
recommendation_list = []

for idx, product in enumerate( sorted_rules["antecedents"] ):
    # antecendent tuple
    for i in list( product ):
        if i == product_id:
            # indexi ne ise (idx) consequentte
            recommendation_list.append( list( sorted_rules.iloc[idx]["consequents"] )[0] )
            recommendation_list = list( dict.fromkeys( recommendation_list ) )
            
            
list_top5 = recommendation_list[0:5]
list_top5          

for elem in list_top5:
    print( check_produto_id( df_country, elem ))  

(64, 'SET/6 RED SPOTTY PAPER PLATES')
(171, 'SET/6 RED SPOTTY PAPER CUPS')


# 7.0  Function Recomendation System 

In [103]:
# ================ Data Preparation ================
def data_preparation( dataframe ):
    # Rename Columns  
    cols_old = ['BillNo', 'ItemName', 'Quantity', 'Date', 'Price', 'CustomerID', 'Country']

    snakecase = lambda x: inflection.underscore( x )
    cols_news = list( map( snakecase, cols_old ) )

    # Rename
    dataframe.columns = cols_news
    
    # Drop NA
    dataframe = dataframe.dropna( subset=['item_name','customer_id'] )
      
    #feature_engineering
    dataframe = dataframe.loc[dataframe['price'] >= 0.04,:]

    dataframe = dataframe[~dataframe["item_name"].str.contains( "POST", na=False )]
    
    dataframe = dataframe[~dataframe['country'].isin( ["Unspecified"] )]

    dataframe = dataframe[~dataframe['customer_id'].isin( [16446] )]

    # data product_id 
    df_product_id = dataframe.drop( ['bill_no', 'quantity', 'date', 'price','customer_id', 'country'], axis=1 ).drop_duplicates( ignore_index=True)
    df_product_id = pd.DataFrame( df_product_id ) 
    df_product_id['produto_id'] = pd.factorize( df_product_id['item_name'])[0]

    # merge 
    dataframe = pd.merge( dataframe, df_product_id, on='item_name', how='left' )
    
    return dataframe

# ================ Filtragem Country ================
def data_filter( dataframe, country=False, Country="" ):
    if country:
        dataframe = dataframe[dataframe["country"] == Country]
    return dataframe

# ============ Tabela Purchase Product ============
def create_purchase_product( dataframe, id=False ):
    if id:
        return dataframe.groupby( ['bill_no', "produto_id"] )['quantity'].sum().unstack().fillna(0). \
            applymap( lambda x: 1 if x > 0 else 0 )
    else:
        return dataframe.groupby( ['bill_no', 'item_name'] )['quantity'].sum().unstack().fillna(0). \
            applymap( lambda x: 1 if x > 0 else 0 ) 
    
# ================ Check Produto ID ================
def check_produto_id( dataframe, produto_id ):
    product_name = dataframe[dataframe["produto_id"] == produto_id]["item_name"].unique()[0]
    return produto_id, product_name

# =========  Apriori Algorithm & ARL Rules ========= 
def apriori_alg( dataframe, support_val=0.06 ):
    inv_pro_df = create_purchase_product( dataframe, id=True )
    frequent_itemsets = apriori( inv_pro_df, min_support=support_val, use_colnames=True )
    rules = association_rules( frequent_itemsets, metric="support", min_threshold=support_val )
    sorted_rules =  rules.sort_values( "support", ascending=False ) 
    return sorted_rules
    
# ================ Rcommend Product ================         
def recommend_product( dataframe, product_id, support_val= 0.06, num_of_products=5 ):
    sorted_rules = apriori_alg( dataframe, support_val )
    recommendation_list = []  
    for idx, product in enumerate( sorted_rules["antecedents"] ):
        for j in list( product ):
            if j == product_id:
                recommendation_list.append( list( sorted_rules.iloc[idx]["consequents"] )[0] )
                recommendation_list = list( dict.fromkeys(recommendation_list) )
    return( recommendation_list[0:num_of_products] )

In [108]:
def recommendation_system( dataframe,support_val=0.01, num_of_products= 5 ):
    product_id = input( "Insira o ID produto: ")
    
    if product_id in list( dataframe["produto_id"].astype("str").unique() ):
        product_list = recommend_product( dataframe, int(product_id), support_val, num_of_products )
        if len( product_list) == 0:
            print( "Não há recomendação para este produto!" )
        else:
            print( "Produtos relacionados com ID do produto:" , product_id , "podem ser vistos abaixo:" )
        
            for i in range( 0, len( product_list[0:num_of_products] ) ):
                print( check_produto_id(dataframe, product_list[i] ) )
            
    else:
        print( "ID do produto inválido, tente novamente!" )

### Loading Data

In [91]:
df_raw = pd.read_excel( '../data/raw/DataSet_Test.xlsx', usecols="A:G" )

### Data Preparation

In [92]:
df1 = df_raw.copy()

df1 = data_preparation( df1 )
df_country = data_filter( df1, True ,'Germany' )
df_country.head()

,bill_no,item_name,quantity,date,price,customer_id,country,produto_id
1095,536527,SET OF 6 T-LIGHTS SANTA,6,2010-12-01 13:04:00,2.95,12662.0,Germany,244
1096,536527,ROTATING SILVER ANGELS T-LIGHT HLDR,6,2010-12-01 13:04:00,2.55,12662.0,Germany,317
1097,536527,MULTI COLOUR SILVER T-LIGHT HOLDER,12,2010-12-01 13:04:00,0.85,12662.0,Germany,634
1098,536527,5 HOOK HANGER MAGIC TOADSTOOL,12,2010-12-01 13:04:00,1.65,12662.0,Germany,150
1099,536527,3 HOOK HANGER MAGIC GARDEN,12,2010-12-01 13:04:00,1.95,12662.0,Germany,635


**Vamos lidar com os dados de vendas da Germany**

### Recommendation System

In [109]:
recommendation_system( df_country )

Insira o ID produto: 171


/opt/anaconda/envs/manfing/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Produtos relacionados com ID do produto: 171 podem ser vistos abaixo:
(64, 'SET/6 RED SPOTTY PAPER PLATES')
(170, 'SET/20 RED RETROSPOT PAPER NAPKINS')
(68, 'PACK OF 72 RETROSPOT CAKE CASES')
(361, 'RED RETROSPOT PLATE')
(603, 'PACK OF 20 NAPKINS PANTRY DESIGN')


In [110]:
recommendation_system( df_country )

Insira o ID produto: 9


/opt/anaconda/envs/manfing/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Não há recomendação para este produto!


In [111]:
recommendation_system( df_country )

Insira o ID produto: 64


/opt/anaconda/envs/manfing/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Produtos relacionados com ID do produto: 64 podem ser vistos abaixo:
(171, 'SET/6 RED SPOTTY PAPER CUPS')
(170, 'SET/20 RED RETROSPOT PAPER NAPKINS')
(68, 'PACK OF 72 RETROSPOT CAKE CASES')
(34, 'ROUND SNACK BOXES SET OF4 WOODLAND')
(1096, 'PACK OF 6 SKULL PAPER PLATES')


In [112]:
recommendation_system( df_country )

Insira o ID produto: 500
ID do produto inválido, tente novamente!


In [113]:
recommendation_system( df_country )

Insira o ID produto: 34


/opt/anaconda/envs/manfing/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Produtos relacionados com ID do produto: 34 podem ser vistos abaixo:
(356, 'ROUND SNACK BOXES SET OF 4 FRUITS')
(423, 'PLASTERS IN TIN WOODLAND ANIMALS')
(35, 'SPACEBOY LUNCH BOX')
(1215, 'WOODLAND CHARLOTTE BAG')
(685, 'PLASTERS IN TIN CIRCUS PARADE')


Aqui estão os resultados das **recomendações**. O Algoritmo sugeri combinação de produtos dentro das cestas de compras dos clientes analisados.

Através dessas informações pode-se criar Campanhas e Técnicas de Vendas: Cross Selling / Descontos. 